In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('F:\\python\\udemi\\pack\\Machine Learning avec Python La formation complete 2023\\3 - Introduction aux K plus proches voisins k Nearest Neigbors kNN\\pratique\\REGRESSION_LIN\\data\\AmesHousing.txt',delimiter = "\t")
train = data.iloc[:1460]
test = data[1460:]

In [3]:
# train sans valeur manquante
train_null_counts = train.isnull().sum()
df_no_mv = train[train_null_counts[train_null_counts == 0].index]

df_no_mv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 58 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Order            1460 non-null   int64  
 1   PID              1460 non-null   int64  
 2   MS SubClass      1460 non-null   int64  
 3   MS Zoning        1460 non-null   object 
 4   Lot Area         1460 non-null   int64  
 5   Street           1460 non-null   object 
 6   Lot Shape        1460 non-null   object 
 7   Land Contour     1460 non-null   object 
 8   Utilities        1460 non-null   object 
 9   Lot Config       1460 non-null   object 
 10  Land Slope       1460 non-null   object 
 11  Neighborhood     1460 non-null   object 
 12  Condition 1      1460 non-null   object 
 13  Condition 2      1460 non-null   object 
 14  Bldg Type        1460 non-null   object 
 15  House Style      1460 non-null   object 
 16  Overall Qual     1460 non-null   int64  
 17  Overall Cond  

## convertir des colonnes à un type de donné catégorique

In [4]:
train["House Style"].value_counts()

House Style
1Story    743
2Story    440
1.5Fin    160
SLvl       60
SFoyer     35
2.5Unf     11
1.5Unf      8
2.5Fin      3
Name: count, dtype: int64

In [5]:
train['Utilities'] = train['Utilities'].astype('category')

In [6]:
## il faut faire comme ceci pour acceder aux valeurs réeles du code:
train['Utilities'].cat.codes

0       0
1       0
2       0
3       0
4       0
       ..
1455    0
1456    0
1457    0
1458    0
1459    0
Length: 1460, dtype: int8

#### pratique

In [7]:
text_cols = df_no_mv.select_dtypes(include=['object']).columns

for col in text_cols:
    print(col+':',len(train[col].unique()))

MS Zoning: 6
Street: 2
Lot Shape: 4
Land Contour: 4
Utilities: 3
Lot Config: 5
Land Slope: 3
Neighborhood: 26
Condition 1: 9
Condition 2: 6
Bldg Type: 5
House Style: 8
Roof Style: 6
Roof Matl: 5
Exterior 1st: 14
Exterior 2nd: 16
Exter Qual: 4
Exter Cond: 5
Foundation: 6
Heating: 6
Heating QC: 4
Central Air: 2
Electrical: 4
Kitchen Qual: 5
Functional: 7
Paved Drive: 3
Sale Type: 9
Sale Condition: 5


In [8]:
for col in text_cols:
    train[col] = train[col].astype('category')
train['Utilities'].cat.codes.value_counts()

0    1457
2       2
1       1
Name: count, dtype: int64

### technique pour gérer les caractéristiques de catégorie colonnes factices

In [9]:
# pd.get_dummies(train/test['nom_colonne'])

for col in text_cols:
    col_dummies = pd.get_dummies(train[col]).astype(int)
    train = pd.concat([train, col_dummies], axis=1)
    del(train[col])
train.head()

,Order,PID,MS SubClass,Lot Frontage,Lot Area,Alley,Overall Qual,Overall Cond,Year Built,Year Remod/Add,...,ConLI,ConLw,New,Oth,WD,Abnorml,Alloca,Family,Normal,Partial
0,1,526301100,20,141.0,31770,NaN,6,5,1960,1960,...,0,0,0,0,1,0,0,0,1,0
1,2,526350040,20,80.0,11622,NaN,5,6,1961,1961,...,0,0,0,0,1,0,0,0,1,0
2,3,526351010,20,81.0,14267,NaN,6,6,1958,1958,...,0,0,0,0,1,0,0,0,1,0
3,4,526353030,20,93.0,11160,NaN,7,5,1968,1968,...,0,0,0,0,1,0,0,0,1,0
4,5,527105010,60,74.0,13830,NaN,5,5,1997,1998,...,0,0,0,0,1,0,0,0,1,0


## transformer des caratéristiques numériques inapropriées

In [10]:
train[["Year Remod/Add","Year Built"]]

,Year Remod/Add,Year Built
0,1960,1960
1,1961,1961
2,1958,1958
3,1968,1968
4,1998,1997
...,...,...
1455,2000,2000
1456,2001,2001
1457,2000,1999
1458,1999,1998


In [11]:
train["years_until_remod"] = train["Year Remod/Add"] - train["Year Built"]

#  gestion des valeurs manquantes

In [12]:
data = pd.read_csv('F:\\python\\udemi\\pack\\Machine Learning avec Python La formation complete 2023\\3 - Introduction aux K plus proches voisins k Nearest Neigbors kNN\\pratique\\REGRESSION_LIN\\data\\AmesHousing.txt',delimiter = "\t")
train = data.iloc[:1460]
test = data[1460:]

In [13]:
# defenir un seul pour la suppresion des colonnes contenant des valeurs manquantes
missing_value = train.isnull().sum()
missing_value.sum()

df_missing_values = train[missing_value[(missing_value>0) & (missing_value<584)].index]

print(df_missing_values.isnull().sum())
print()
print(df_missing_values.dtypes) 

Lot Frontage      249
Mas Vnr Area       11
Bsmt Qual          40
Bsmt Cond          40
Bsmt Exposure      41
BsmtFin Type 1     40
BsmtFin SF 1        1
BsmtFin Type 2     41
BsmtFin SF 2        1
Bsmt Unf SF         1
Total Bsmt SF       1
Bsmt Full Bath      1
Bsmt Half Bath      1
Garage Type        74
Garage Yr Blt      75
Garage Finish      75
Garage Qual        75
Garage Cond        75
dtype: int64

Lot Frontage      float64
Mas Vnr Area      float64
Bsmt Qual          object
Bsmt Cond          object
Bsmt Exposure      object
BsmtFin Type 1     object
BsmtFin SF 1      float64
BsmtFin Type 2     object
BsmtFin SF 2      float64
Bsmt Unf SF       float64
Total Bsmt SF     float64
Bsmt Full Bath    float64
Bsmt Half Bath    float64
Garage Type        object
Garage Yr Blt     float64
Garage Finish      object
Garage Qual        object
Garage Cond        object
dtype: object


In [15]:
## imputation 
# pour les types "float"
"""missing_float = df_missing_values.select_dtypes(include=['float']).columns

fill_with_zero = missing_float.fillna(0)
#ou
fill_with_mean = missing_float.fillna(missing_float.mean())  """

float_cols = df_missing_values.select_dtypes(include=['float'])

float_cols = float_cols.fillna(float_cols.mean())

float_cols.isnull().sum()

Lot Frontage      0
Mas Vnr Area      0
BsmtFin SF 1      0
BsmtFin SF 2      0
Bsmt Unf SF       0
Total Bsmt SF     0
Bsmt Full Bath    0
Bsmt Half Bath    0
Garage Yr Blt     0
dtype: int64